In [1]:
"""

# Note, in Colab that we use the UI to mount G-drive

"""

import os
from pathlib import Path
print("Loading VPN 30s (ftm) ds.")
data_dir30 = Path(os.path.abspath("/content/drive/MyDrive/eep567/TimeBasedFeatures-Dataset-30s-VPN.arff"))

import pandas as pd
df = pd.read_csv(data_dir30)
####df.head()
####df.info()
####df["Unnamed: 23"].value_counts()
####df.describe() #(shows max of 1000000 and min of 0 in Unnamed: 9; large variance in values)
proj_random_seed = 42
recursion_depth = df.shape[1] // 2    #(experimental depth as half of # columns)

# derive feature / label subgroups from pool
# (skip the top 26 rows which are NaN)
# the "Unnamed: 23" column contains the network traffic category
features_pool = df.drop(["Unnamed: 23"], axis=1).values[26:]
category_pool = df["Unnamed: 23"].values[26:]
from sklearn.model_selection import train_test_split
train_set, test_set, train_category, test_category = train_test_split(
    features_pool, category_pool, test_size=0.2, random_state=proj_random_seed
)
print("Train set size:", len(train_category))
print("Test set size:", len(test_category))

# for decision trees, scaling is unnecessary. only use scaling with PCA
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
pca_pipeline = make_pipeline(StandardScaler(), PCA())
features_std_scaled = pca_pipeline.fit_transform(train_set)

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

ohenc = OneHotEncoder(handle_unknown='ignore')
many_observations_of_one_feature = train_category.reshape(-1,1)
fitted_categories = ohenc.fit(many_observations_of_one_feature)
# ~ print(fitted_categories.categories_)
# ~ print(many_observations_of_one_feature[:10])

labels_train = ohenc.transform(many_observations_of_one_feature).toarray()
# regularize with max_depth
dtc_model = DecisionTreeClassifier(max_depth=recursion_depth, random_state=proj_random_seed)
dtc_model.fit(features_std_scaled, labels_train)

# to inspect the tree (as plain text)
from sklearn import tree
tree_format_text = tree.export_text(dtc_model)
# ~ print(tree_format_text)
# to inspect the tree (as graphvis diagram)
# ~ category_columns = ohenc.get_feature_names_out()
# ~ tree.export_graphviz(
	# ~ dtc_model,
	# ~ out_file="timeout30_tree.dot",
	# ~ class_names=category_columns,
	# ~ rounded=True,
	# ~ filled=True
# ~ )
# ~ from graphviz import Source
# ~ Source.from_file("timeout30_tree.dot")


# ~ from sklearn.model_selection import cross_val_score
# ~ measure_performance = cross_val_score(dtc_model, features_std_scaled, labels_train , cv=10)
# ~ pd.Series(measure_performance).describe()
from sklearn.model_selection import cross_val_predict
# ~ labels_predict = cross_val_predict(dtc_model, features_std_scaled, labels_train , cv=10)
# ~ decode_predict = labels_predict.argmax(axis=1)
# ~ decode_train = labels_train.argmax(axis=1)
# ~ from sklearn.metrics import confusion_matrix
# ~ cm = confusion_matrix(decode_train, decode_predict)
# ~ print(cm)
# ~ from sklearn.metrics import precision_score, recall_score
# ~ pr_score = precision_score(decode_train, decode_predict, average='weighted')
# ~ rc_score = recall_score(decode_train, decode_predict, average='weighted')
# ~ print(pr_score)
# ~ print(rc_score)
# ~ from sklearn.metrics import f1_score
# ~ combo_score = f1_score(decode_train, decode_predict, average='weighted')
# ~ print(combo_score)

n_observations_test_category = test_category.reshape(-1,1)
labels_test = ohenc.transform(n_observations_test_category).toarray()
features_scaled_test = pca_pipeline.fit_transform(test_set)

labels_predict_test = cross_val_predict(dtc_model, features_scaled_test, labels_test , cv=10)
decode_predict_test = labels_predict_test.argmax(axis=1)
decode_test = labels_test.argmax(axis=1)
from sklearn.metrics import classification_report
classifi_rpt = classification_report(decode_test, decode_predict_test)
print(classifi_rpt)

# nearest neighbor algo
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(features_std_scaled, labels_train)
knn_train_predict = cross_val_predict(knn_clf, features_std_scaled, labels_train , cv=10)
from sklearn.metrics import f1_score
# macro means all labels are equally important (versus weighted)
knn_f1_score = f1_score(labels_train, knn_train_predict, average='macro')
print("KNN F1:", knn_f1_score)
from sklearn.metrics import precision_score, recall_score
knn_pr_score = precision_score(labels_train, knn_train_predict, average='macro')
knn_rc_score = recall_score(labels_train, knn_train_predict, average='macro')
print("KNN precision:", knn_pr_score)
print("KNN recall:", knn_rc_score)
import numpy as np
# we are specifying np.nan in the report to address the warning of unpredicted labels
knn_validat_predict = cross_val_predict(knn_clf, features_scaled_test, labels_test , cv=10)
# ~ knn_classifi_rpt = classification_report(labels_test, knn_validat_predict)
knn_classifi_rpt = classification_report(labels_test, knn_validat_predict, zero_division=np.nan)
print("KNN report:\n", knn_classifi_rpt)




Loading VPN 30s (ftm) ds.
Train set size: 6187
Test set size: 1547
              precision    recall  f1-score   support

           0       0.77      0.85      0.81       528
           1       0.51      0.44      0.47       149
           2       0.61      0.51      0.56       205
           3       0.82      0.82      0.82       175
           4       0.75      0.74      0.75       175
           5       0.79      0.78      0.79        59
           6       0.96      0.96      0.96       256

    accuracy                           0.77      1547
   macro avg       0.74      0.73      0.74      1547
weighted avg       0.76      0.77      0.76      1547

KNN F1: 0.7989104467470252
KNN precision: 0.8284589045754632
KNN recall: 0.7766845815825748
KNN report:
               precision    recall  f1-score   support

           0       0.78      0.85      0.81       528
           1       0.55      0.32      0.41       149
           2       0.70      0.40      0.51       205
           3  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
